In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

import sys
sys.path.insert(1, "./code/")
from Utils import Utils


/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peterkong/m

In [2]:
train_df = pd.read_csv("./data/train.csv")

In [11]:
# sanity checks
#train_df.id.nunique()
# 19579

#train_df.author.unique()
# array(['EAP', 'HPL', 'MWS'], dtype=object)

array(['EAP', 'HPL', 'MWS'], dtype=object)

In [3]:
Utils.check_for_nulls(train_df)

{'author': 0, 'id': 0, 'text': 0}

In [14]:
import spacy
import nltk
from nltk.corpus import stopwords

nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')

def stem_n_stop(text, spacy):
    wordlist = nlp(text)
    
    
    
import sys
#sys.path.insert(1, "./feature-extraction/embed-extractor/")
#from EmbedExtractor import EmbedExtractor
from FileReader import FileReader

    

(19579, 3)

Index(['id', 'text', 'author'], dtype='object')